# Loading the Data

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
wec=pd.read_csv("C:\\Project DATA\\WEC Analysis 2021\\Data\\wec_hypercar_2021.csv")
Bahrain_6=wec[wec['circuit']=='BAHRAIN INTERNATIONAL CIRCUIT 6 HOURS']
Bahrain_6["pit_time"] = pd.to_timedelta(Bahrain_6["pit_time"], errors="coerce")
Bahrain_6["pit_time_sec"] = Bahrain_6["pit_time"].dt.total_seconds()

C:\Users\User\AppData\Local\Temp\ipykernel_25072\1650108838.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bahrain_6["pit_time"] = pd.to_timedelta(Bahrain_6["pit_time"], errors="coerce")
C:\Users\User\AppData\Local\Temp\ipykernel_25072\1650108838.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bahrain_6["pit_time_sec"] = Bahrain_6["pit_time"].dt.total_seconds()


# Analysis

## Driver Stint Consistency

In [3]:
fig = px.line(Bahrain_6,
    x="lap_number",
    y="lap_time_s",
    title="Driver Stint Consistency",
    color="team_no",          
    line_group="driver_stint", 
    markers=True,
    labels={                  
        "lap_number": "Lap Number",
        "lap_time_s": "Lap Time (s)",
        "driver_stint": "Driver Stint",
        "team_no": "Team Number"
    }
)

fig.update_layout(
    xaxis_title="Lap Number",
    yaxis_title="Lap Time (s)",
    legend_title="Team Number"
)

fig.show()


In [5]:
fig = px.line(Bahrain_6,
    x="lap_number",
    y="lap_time_s",
    title="Driver Stint Consistency",
    color="driver_stint",
    line_group="team_no",
    markers=True,
    labels={                  
        "lap_number": "Lap Number",
        "lap_time_s": "Lap Time (s)",
        "driver_stint": "Driver Stint",
        "team_no": "Team Number"
    }
)

fig.update_layout(
    xaxis_title="Lap Number",
    yaxis_title="Lap Time (s)",
    legend_title="Driver Name"
)

fig.show()


## Pit Stop Strategy Effictiveness 

In [3]:
import plotly.graph_objects as go

fig = go.Figure()

for team in Bahrain_6["team_no"].unique():
    team_df = Bahrain_6[Bahrain_6["team_no"] == team]

    fig.add_trace(go.Scatter(
        x=team_df["lap_number"],
        y=team_df["lap_time_s"],
        mode="lines",
        line=dict(shape="hv"),
        name=f"Team {team}",
        legendgroup=str(team),  
        showlegend=True
    ))

    pit_laps = team_df[team_df["pit_time_sec"].notna() & (team_df["pit_time_sec"] > 0)]
    fig.add_trace(go.Scatter(
        x=pit_laps["lap_number"],
        y=pit_laps["lap_time_s"],
        mode="markers",
        marker=dict(size=10, symbol="x", color="red"),
        name="Pit Stop",
        legendgroup=str(team),  
        showlegend=False        
    ))

fig.update_layout(
    title="Pit Stop Strategy Effectiveness",
    xaxis_title="Lap Number",
    yaxis_title="Lap Time (s)",
    legend_title="Team Number"
)

fig.show()


## Stint Length vs Degradation

In [7]:
Bahrain_6["stint_lap_index"] = Bahrain_6.groupby("driver_stint").cumcount()

first_lap_time = (Bahrain_6[Bahrain_6["stint_lap_index"] == 0].set_index("driver_stint")["lap_time_s"])

Bahrain_6["lap_time_delta"] = (Bahrain_6["lap_time_s"] - Bahrain_6["driver_stint"].map(first_lap_time))

fig = px.density_heatmap(Bahrain_6,
    x="lap_number",
    y="driver_stint",
    z="lap_time_delta",
    histfunc="avg",
    color_continuous_scale="RdYlBu_r",
    title="Stint Length vs Lap Time Degradation (Delta vs First Lap)",
     labels={                  
        "lap_number": "Lap Number",
        "lap_time_delta": "Lap Time Δ (s)",
        "driver_stint": "Driver Stint",
    }
)

fig.update_layout(
    xaxis_title="Lap Number",
    yaxis_title="Driver Stint",
    coloraxis_colorbar_title="Lap Time Δ (s)"
)

fig.update_xaxes(range=[1, 185])

fig.show()

C:\Users\User\AppData\Local\Temp\ipykernel_13608\1667256836.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\User\AppData\Local\Temp\ipykernel_13608\1667256836.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
stint_summary = (
    Bahrain_6.groupby(["driver_stint", "team_no"]).agg(
        start_lap=("lap_number", "min"),
        end_lap=("lap_number", "max"),
        start_time=("lap_time_s", "first"),
        end_time=("lap_time_s", "last")
    )
    .reset_index()
)

stint_long = pd.concat(
    [stint_summary[["driver_stint", "team_no", "start_lap", "start_time"]].rename(columns={"start_lap": "lap", "start_time": "lap_time"}),

    stint_summary[["driver_stint", "team_no", "end_lap", "end_time"]].rename(columns={"end_lap": "lap", "end_time": "lap_time"})],
    ignore_index=True
)

fig = px.line(stint_long,
    x="lap",
    y="lap_time",
    color="driver_stint",   
    line_group="team_no", 
    markers=True,
    title="Stint Degradation: Start vs End",
    labels={                  
        "lap": "Lap Number",
        "lap_time": "Lap Time",
        "driver_stint": "Driver Stint",
        "team_no": "Team Number"
    }
)

fig.update_layout(
    xaxis_title="Lap Number",
    yaxis_title="Lap Time (s)"
)

fig.show()

In [4]:
stint_summary = (
    Bahrain_6.groupby("driver_stint")
    .agg(
        start_lap_time=("lap_time_s", "first"),
        end_lap_time=("lap_time_s", "last"),
        stint_length=("lap_time_s", "count"),
        team_no=("team_no", "first"), 
        driver=("driver_name", "first")
    )
    .reset_index()
)

stint_summary["delta_end_start"] = (stint_summary["end_lap_time"] - stint_summary["start_lap_time"])

fig = px.scatter(
    stint_summary,
    x="stint_length",
    y="delta_end_start",
    color="team_no",
    hover_data=["driver", "driver_stint"],
    title="Stint Degradation vs Stint Length",
    labels={                  
        "team_no": "Team Number",
        "stint_length": "Stint Length",
        "driver_stint": "Driver Stint",
        "team_no": "Team Number",
        "driver": "Driver",
        "delta_end_start": "Delta End Start"
    }
)

fig.update_layout(
    xaxis_title="Stint Length (laps)",
    yaxis_title="Lap Time Degradation (End − Start) [s]"
)

fig.show()



## Gap Evolution to Leader

In [6]:
fig = px.line(Bahrain_6,
    x="lap_number",
    y="class_gap",
    color="team_no",
    title="Gap Evolution to Race Leader",
    labels={
        "lap_number": "Lap Number",
        "class_gap": "Gap to Leader (s)",
        "team_no": "Team Number"
    }
)

fig.show()

## Drive Pace Delta within Team and Team Pace Delta

In [5]:
Bahrain_6["stint_lap_index"] = Bahrain_6.groupby("driver_stint").cumcount()

first_lap_time = (Bahrain_6[Bahrain_6["stint_lap_index"] == 0].set_index("driver_stint")["lap_time_s"])

Bahrain_6["lap_time_delta"] = (Bahrain_6["lap_time_s"] - Bahrain_6["driver_stint"].map(first_lap_time))

fig = px.box(Bahrain_6,
    x="team_no",
    y="lap_time_delta",
    color="driver_name",
    title="Driver Pace Delta Within Team"
)

fig.update_layout(
    xaxis_title="Team",
    yaxis_title="Lap Time Δ vs Stint First Lap (s)"
)

fig.show()


C:\Users\User\AppData\Local\Temp\ipykernel_25072\3004173932.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\User\AppData\Local\Temp\ipykernel_25072\3004173932.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
Bahrain_6["stint_lap_index"] = Bahrain_6.groupby("driver_stint").cumcount()

first_lap_time = (Bahrain_6[Bahrain_6["stint_lap_index"] == 0].set_index("driver_stint")["lap_time_s"])

Bahrain_6["lap_time_delta"] = (Bahrain_6["lap_time_s"] - Bahrain_6["driver_stint"].map(first_lap_time))

fig = px.violin(Bahrain_6,
    x="team_no",
    y="lap_time_delta",
    box=True,          
    points="outliers", 
    color="team_no",
    title="Team Pace Comparison (Lap Time Δ vs Stint First Lap)"
)

fig.update_layout(
    xaxis_title="Team",
    yaxis_title="Lap Time Δ (s)",
    showlegend=False
)

fig.show()


C:\Users\User\AppData\Local\Temp\ipykernel_11536\4083104374.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\User\AppData\Local\Temp\ipykernel_11536\4083104374.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

